In [ ]:
import pandas as pd
import os
import numpy as np
import cv2
#from tqdm import tqdm
import matplotlib .pyplot as plt
import keras

In [ ]:
train=pd.read_csv("C:\\python\\image\\train1.csv")

test=pd.read_csv("C:\\python\\image\\test.csv")
train.head(2)

In [ ]:
# Convert categorical to numerical for training model
class_map = {'Food':0, 'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food', 1:'Attire',2: 'Decorationandsignage',3 :'misc'}
train['Class']=train['Class'].map(class_map)


In [ ]:
train['Class']

In [ ]:
# Encoded all class to numerical values
train.head(2)

In [ ]:
train_img=[]
train_label=[]
j=0
path="C:\\python\\image\\Train Images"
for i in (train["Image"]):
  final_path=os.path.join(path,i)
  img=cv2.imread(final_path)
  img=cv2.resize(img,(150,150))
  img=img.astype('float32')
  train_img.append(img)
  train_label.append(train['Class'][j])
  j=j+1




In [ ]:
(h, w, d) = img.shape
print("width={}, height={}, depth={}".format(w, h, d))
cv2.imshow("Image", train_img[0])
#cv2.waitKey(0)


In [ ]:
train.head(10)

In [ ]:
#Image agumentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(train_img)

In [ ]:
# Test_Image
test_img=[]


path="C:\\python\\image\\Test Images"
for i in (test["Image"]):
  final_path=os.path.join(path,i)
  img=cv2.imread(final_path)
  img=cv2.resize(img,(150,150))
  img=img.astype('float32')
  test_img.append(img)
  


In [ ]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

In [ ]:
from keras.applications.inception_v3 import InceptionV3

#from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
#from tensorflow.keras.models import Model,Sequential

from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# InceptionV3 model
base_model1 = InceptionV3(input_shape=(150,150,3), weights='imagenet', pooling='avg',include_top=False)


model1=Sequential()
model1.add(base_model1)

model1.add(Dense(256,activation='relu'))
model1.add(Dense(4,activation='softmax'))


from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

base_model1.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.000001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model1.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model1.fit_generator(datagen.flow(train_img, to_categorical(train_label,4), batch_size=32),
                    epochs=20,callbacks=callbacks)

In [ ]:
print(keras.__version__)

In [ ]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })


In [ ]:
submission.head(10)


In [ ]:
submission.to_csv('C:\\python\\image\\submission.csv', index=False)